In [ ]:
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp
from qiskit.visualization import plot_distribution
from qiskit.primitives import Estimator, Sampler  # Local simulator
from scipy.optimize import minimize

from typing import List, Dict, Tuple

In [ ]:
# Urgelles grafo
# P = 27
GRAPH_NAME = "Urgelles grafo"

NUM_NODES = 4
EDGES = {(0, 1): 5, (0, 2): 8, (1, 2): 2, (1, 3): 7, (2, 3): 4}
NQUBITS = len(EDGES)

# Hp = 11*Z0 - 17.5*Z1 - 28*Z2 - 17*Z3 + 11.5*Z4 +
#      13.5*(-Z0*Z2 + Z1*Z2 + Z2*Z3 - Z2*Z4 + Z0*Z1 - Z0*Z3 - Z1*Z4)

# Coeficientes lineales de Hp (11 * Z0 - 17.5 * Z1...)
LINEAR_COEFS = [11, -17.5, -28, -17, 11.5]

# Coeficientes cuadráticos Ej: {(2, 3): 13.5} -> 13.5 * Z2 * Z3
QUADRA_COEFS = {(0, 1): 13.5, (0, 2): -13.5, (0, 3): -13.5, (1, 2): 13.5,
                (1, 4): -13.5, (2, 3): 13.5, (2, 4): -13.5}

CIRCUIT_ROUTE = "../../latex/img/circuits/primer/primer-circuit-qaoaAnsatz-p1.png"
PRINT_GRAPH_LAYOUT = {0: [-1, 0], 1: [0, 1], 2: [0, -1], 3: [1, 0]}

In [ ]:
# Shan grafo
# P = 20
GRAPH_NAME = "Shan grafo"

NUM_NODES = 4
EDGES = {(0, 1): 3, (0, 2): 6, (1, 3): 9, (2, 3): 1}
NQUBITS = len(EDGES)

# Hp = - 1.5*Z0 - 3*Z1 - 4.5*Z2 - 0.5*Z3 +
#        10*Z0*Z1 - 10*Z0*Z2 - 10*Z1*Z3 + 10*Z2*Z3

LINEAR_COEFS = [-1.5, -3, -4.5, -0.5]
QUADRA_COEFS = {(0, 1): 10, (0, 2): -10, (1, 3): -10, (2, 3): 10}

CIRCUIT_ROUTE = "../../latex/img/circuits/zhiqiang/zhiqiang-circuit-qaoaAnsatz-p1.png"
PRINT_GRAPH_LAYOUT = {0: [-1, 0], 1: [0, 1], 2: [0, -1], 3: [1, 0]}

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt


# Imprime un grafo dirigido pesado
# Entrada:
#   - graph_nodes: Lista de nodos del grafo
#   - graph_edges: Diccionario {edge: weight} donde edge es (n1, n2)
#   - layout: Para especificar la posición de los nodos
def print_graph(graph_nodes: List[int],
                graph_edges: Dict[Tuple[int, int], int],
                title="", layout: Dict[int, List[int]]=None):
    G = nx.DiGraph()

    # Nodes
    G.add_nodes_from(graph_nodes)

    # Edges
    for pair, weight in graph_edges.items():
        G.add_edge(pair[0], pair[1], weight=weight)

    # Print graph
    if layout is None:
        layout = nx.spring_layout(G)

    plt.figure(3, figsize=(6, 3))
    plt.title(title)
    nx.draw(G, layout, with_labels=True, node_size=600)
    edge_labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G, pos=layout, edge_labels=edge_labels)
    #plt.savefig("../../latex/img/primer_grafo/primer_grafo.png")


print_graph(range(NUM_NODES), EDGES, title=GRAPH_NAME, layout=PRINT_GRAPH_LAYOUT)

In [ ]:
# Crear lista para SparsePauliOp. Equivalente al operador C
hamiltonian_list = []
for coef, qubit_idx in zip(LINEAR_COEFS, range(len(LINEAR_COEFS))):
    gates = ["I"] * NQUBITS
    gates[(NQUBITS - qubit_idx - 1)] = "Z"
    hamiltonian_list.append(("".join(gates), coef))

for qubit_idxs in QUADRA_COEFS:
    coef = QUADRA_COEFS[qubit_idxs]
    gates = ["I"] * NQUBITS
    gates[(NQUBITS - qubit_idxs[0] - 1)] = "Z"
    gates[(NQUBITS - qubit_idxs[1] - 1)] = "Z"
    hamiltonian_list.append(("".join(gates), coef))

hamiltonian_list

In [ ]:
# Construir hamiltoniano y generar circuito ansatz
hamiltonian = SparsePauliOp.from_list(hamiltonian_list)
num_layers = 1
ansatz = QAOAAnsatz(hamiltonian, reps=num_layers)
ansatz.decompose(reps=3).draw(output="mpl", fold=-1)

In [ ]:
# Función a minimizar con el optimizador clásico


def execute_circuit(params, ansatz, hamiltonian, estimator):
    cost = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    return cost

In [ ]:
# Inicialización de entornos de ejecución
estimator = Estimator(options={"shots": int(1000)})
sampler = Sampler(options={"shots": int(1000)})

In [ ]:
# Calculo de parametros alpha y beta óptimos
x0 = [1.0, 1.0] * num_layers
res = minimize(execute_circuit, x0, args=(ansatz, hamiltonian, estimator), method="COBYLA")
res

In [ ]:
# Cálculo de valor a partir de los parámetros óptimos
qc = ansatz.assign_parameters(res.x)
qc.measure_all()
samp_dist = sampler.run(qc).result().quasi_dists[0]
plot_distribution(samp_dist.binary_probabilities())

In [ ]:
# Cálculo de las estadísticas
# En el TFG:
#   - NA/TE == max_statistics
#   - MM/TE == global_statistics
#
# Entrada:
#   - num_layers: Número de capas
#   - num_generations: TE
#
# Salida:
#   - (max_statistics, global_statistics)
#
def max_global_statistics(num_layers=1, num_generations=100) -> Tuple[Dict[str, float],
                                                                      Dict[str, float]]:
    max_statistics = {}
    global_statistics = {}
    ansatz = QAOAAnsatz(hamiltonian, reps=num_layers)
    x0 = [1.0, 1.0] * num_layers
    for iteration in range(0, num_generations):
        theta_res = minimize(execute_circuit, x0, args=(ansatz, hamiltonian, estimator), method="COBYLA")
        qc = ansatz.assign_parameters(theta_res.x)
        qc.measure_all()
        counts = sampler.run(qc).result().quasi_dists[0].binary_probabilities()

        # Max statistics
        max_path = max(counts, key=counts.get)
        if max_path not in max_statistics:
            max_statistics[max_path] = 0
        max_statistics[max_path] += 1

        # Global statistics
        for (path, num_appearances) in counts.items():
            if path not in global_statistics:
                global_statistics[path] = 0
            global_statistics[path] += num_appearances

    for path in global_statistics:
        global_statistics[path] = global_statistics[path] / num_generations  # Normalize

    for path in max_statistics:
        max_statistics[path] = max_statistics[path] / num_generations  # Normalize

    max_statistics = sorted(max_statistics.items(), key=lambda x: x[1], reverse=True)
    global_statistics = sorted(global_statistics.items(), key=lambda x: x[1], reverse=True)
    return max_statistics, global_statistics

print(GRAPH_NAME + ":\n")
interval_num_layers = (1, 5)
for p in range(interval_num_layers[0], interval_num_layers[1] + 1):
    max_st, global_st = max_global_statistics(num_layers=p, num_generations=100)
    print(f"Max statistics (p = {p}): ", str(max_st))
    print(f"Global statistics (p = {p}): ", str(global_st))
    print()